# Numerical experiments for the RF model

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict, OrderedDict
import matplotlib
from matplotlib.gridspec import GridSpec
import sys
import random
import copy
import sklearn
import skimage
from sklearn.neural_network import MLPRegressor
import itertools
sys.path.append('.')

plt.rcParams.update(plt.rcParamsDefault)
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font',       size=2*MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes',  titlesize=2*MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes',  labelsize=2*MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=2*SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=2*SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=2*SMALL_SIZE)    # legend fontsize
plt.rc('figure',titlesize=2*BIGGER_SIZE)  # fontsize of the figure title
plt.rc("lines", linewidth=3.0)
plt.rcParams["savefig.dpi"] = 500
cmap = plt.get_cmap('Spectral_r')

savedir = './figs/'
datadir = './data/'

In [3]:
def dict_product(d):
    keys = d.keys()
    for element in itertools.product(*d.values()):
        yield dict(zip(keys, element))

# Generate the data

In [4]:
def generate_orthogonal(D, P, eps=1, max_iter=100):
    from scipy.stats import ortho_group
    U = torch.from_numpy(ortho_group.rvs(P)).float()
    V = torch.from_numpy(ortho_group.rvs(D)).float()
    S = torch.zeros((P, D))
    for i in range(min(D,P)):
        S[i,i] = D**0.5 * max(1, (D/P)**0.5)
    points = U @ S @ V
    return points

def generate_rfs(K,D,P,init='gaussian'):
    rfs = []
    for k in range(K):
        if init=='gaussian':
            rf = torch.randn((D,P))
        elif init=='orthogonal':
            rf = generate_orthogonal(D, P).t()
        elif init=='even':
            rf = generate_even(D, P, lr=10, eps=.0, max_iter=10).t()
        rfs.append(rf)
    rf = torch.stack(rfs)
    return rf


def get_dataset(dataset, D=None, N_train=10000, N_test=10000, norm=1, resize_method='PCA', cov=None):
    
    if dataset=='random':
        if cov is None:
            tr_data = torch.randn((N_train,D))
            te_data = torch.randn((N_test,D))
        else:
            tr_data = torch.from_numpy(np.random.multivariate_normal(mean=np.zeros(D),cov=cov,size=(N_train))).float()
            te_data = torch.from_numpy(np.random.multivariate_normal(mean=np.zeros(D),cov=cov,size=(N_test ))).float()

    else:
        import utils
        tr_data, te_data, input_size, output_size = utils.get_data(dataset, d=D, n=N_train)
        if D is not None:
            if resize_method=='PCA':
                tr_data, te_data = utils.get_pca(tr_data, te_data, D, normalized=True)
            else:
                tr_data, te_data = utils.resize_data(tr_data, te_data, D)
        tr_data = tr_data.data.view(tr_data.data.size(0), -1)
        te_data = te_data.data.view(te_data.data.size(0), -1)
                
    return tr_data, te_data
    
def get_act(actname):
    if actname=='relu':
        return lambda x : F.relu(x)
    elif actname=='abs':
        return lambda x : abs(x)
    elif actname=='tanh':
        return lambda x : torch.tanh(x)
    elif actname=='linear':
        return lambda x : x
    elif type(actname)!=str: 
        alpha=actname
        return lambda x : (F.relu(x) + alpha*F.relu(-x) - (1+alpha)/np.sqrt(2*np.pi))/np.sqrt((1+alpha**2)/2 - (1+alpha)**2/(2*np.pi))
    else:
        raise NotImplemented
        
def test(k, act, rf, a, te_data, y_test, task='regression'):
            
    K,D,P=rf.size()

    errors = []
    for ik in range(int(K/k)):
        y_pred = torch.einsum('knp,kp->kn',(act(te_data @ rf[k*ik:k*(ik+1)] / D**0.5) / D**0.5, a[k*ik:k*(ik+1)]))
        if task == 'classification':
            y_pred = y_pred.sign()
        error = (y_pred-y_test).mean(0).pow(2).mean() / 2
        if task == 'classification':
            error /= 2
        errors.append(error)

    return np.mean(errors), np.std(errors)   

def calc_norms(rf, a, X):
    K, D, P = rf.size()
    N, D = X.size()
    norms_curr = []
    for k in range(K):
        original = a[k]
        norm = original.norm().mean().item()/P
        subspace = X @ rf[k] / D
        subspace = subspace / subspace.norm(dim=-1, keepdim=True)
        projected = original @ subspace.t() @ torch.pinverse(subspace @ subspace.t()) @ subspace / max(1, N/P)
        orth = original-projected
        norm         = original .norm().item()/P
        norm_proj    = projected.norm().item()/P
        norm_orth    = orth     .norm().item()/P
        norms_curr.append([norm, norm_proj, norm_orth])
    norms = np.mean(np.array(norms_curr), axis=0)
    return norms

# Train the RF model

In [5]:
def run_rf(args):
    
    D, K = args['D'], args['K']
    rx, rb, rphi = args['rx'], args['rb'], args['rphi']

    cov = torch.zeros((D,D))
    ds = np.array([1/rphi, 1-1/rphi])
    cs = np.array([rx/(rx*ds[0]+ds[1]), 1/(rx*ds[0]+ds[1])])
    betas = np.array([rb/(rb*ds[0]*cs[0]+ds[1]*cs[1]), 1/(rb*ds[0]*cs[0]+ds[1]*cs[1])])

    for i in range(D):
        if i < int(D/rphi) : 
            cov[i,i] = cs[0]
        else : 
            cov[i,i] = cs[1]
    beta = torch.zeros(D)
    beta[:int(D/rphi)] = betas[0]**.5 * torch.randn(int(D/rphi))
    beta[int(D/rphi):] = betas[1]**.5 * torch.randn(D-int(D/rphi))
    
    teacher = lambda x : x @ beta / D**0.5 

    tr_data, te_data = get_dataset(args['dataset'], D, N_train=int(D*max(args['Psi2_list'])), N_test=args['N_test'], cov=cov, resize_method='PCA')
    act = get_act(args['actname'])
    
    tr_errors = []
    tr_errors_std = []
    te_errors = []
    te_errors_std = []
    te_errors_ens = []
            
    y_test = teacher(te_data)
    if args['task']=='classification': y_test = y_test.sign()
            
    for i, Psi1 in enumerate(args['Psi1_list']):
        for j, Psi2 in enumerate(args['Psi2_list']):

            P = int(D * Psi1)
            N = int(D * Psi2)
            rf = generate_rfs(K,D,P,init='gaussian')

            X = tr_data[:N]
            y = teacher(X) 
            if args['task']=='classification': 
                y = y.sign()
                y *= ((np.random.uniform(0,1,len(y))>args['tau']).astype(int)-.5)*2
                y = y.float()
            else: y += args['tau']**.5 * torch.randn((N,))

            a = torch.empty((K,P))

            for k in range(K):
                Z = act(X @ rf[k] / D**0.5) / D**0.5
                a[k] = y @ Z @ torch.from_numpy(np.linalg.pinv((Z.t() @ Z + .5 * args['lam'] * torch.eye(P)).numpy()))

            tr_error, tr_error_std         = test(1, act, rf, a, X, y           , task=args['task'])
            te_error, te_error_std         = test(1, act, rf, a, te_data, y_test, task=args['task'])
            te_error_ens, te_error_ens_std = test(K, act, rf, a, te_data, y_test, task=args['task'])

            tr_errors.append(tr_error)
            tr_errors_std.append(tr_error_std)
            te_errors.append(te_error)
            te_errors_std.append(te_error_std)
            te_errors_ens.append(te_error_ens)
            
        
    run = {'args':copy.deepcopy(args),
           'tr_error':tr_errors,
           'tr_error_std':tr_errors_std,
           'te_error':te_errors,
           'te_error_std':te_errors_std,
           'te_error_ens':te_errors_ens}
    return run

# Train both layers

In [ ]:
def run_nn(args):
    
    D, K = args['D'], args['K']
    rx, rb, rphi = args['rx'], args['rb'], args['rphi']

    cov = torch.zeros((D,D))
    ds = np.array([1/rphi, 1-1/rphi])
    cs = np.array([rx/(rx*ds[0]+ds[1]), 1/(rx*ds[0]+ds[1])])
    betas = np.array([rb/(rb*ds[0]*cs[0]+ds[1]*cs[1]), 1/(rb*ds[0]*cs[0]+ds[1]*cs[1])])

    for i in range(D):
        if i < int(D/rphi) : 
            cov[i,i] = cs[0]
        else : 
            cov[i,i] = cs[1]
    beta = torch.zeros(D)
    beta[:int(D/rphi)] = betas[0]**.5 * torch.randn(int(D/rphi))
    beta[int(D/rphi):] = betas[1]**.5 * torch.randn(D-int(D/rphi))
    
    teacher = lambda x : x @ beta / D**0.5 

    tr_data, te_data = get_dataset(args['dataset'], D, N_train=int(D*max(args['Psi2_list'])), N_test=args['N_test'], cov=cov)
    act = get_act(args['actname'])
    
    tr_errors = []
    te_errors = []
    te_errors_std = []
            
    y_test = teacher(te_data)
            
    for i, Psi1 in enumerate(args['Psi1_list']):
        for j, Psi2 in enumerate(args['Psi2_list']):
            #print(i,j)

            P = int(D * Psi1)
            N = int(D * Psi2)

            X = tr_data[:N]
            y = teacher(X) + args['tau']**.5 * torch.randn((N,))
            
            tmp = []
            for k in range(args['K']):
                model = MLPRegressor(activation=args['actname'], hidden_layer_sizes=[P], solver='adam', alpha=0, learning_rate='constant', max_iter=500)
                model.fit(X,y)
                y_pred = model.predict(te_data)
                y_true = teacher(te_data)
                te_error = np.mean((y_pred-y_true.numpy())**2)
                tmp.append(te_error)
                
            te_errors.append(np.mean(tmp))            
            te_errors_std.append(np.std(tmp))            
        
    run = {'args':copy.deepcopy(args),
           'te_error':te_errors,
           'te_error_std':te_errors}
    return run

# Run the code

In [ ]:
args = {'D' :150,
    'Psi2_list'  : [1],
    'Psi1_list' : np.logspace(-1,1,19),
    'N_test' : 5000,
    'K' : 50,
    'lam' : 1e-3,
    'actname': 'relu',
    'dataset' : 'random',
    'rphi': 10}

model = "rf"

constraints = {'tau':[0.3],
       'task':['classification']}

for cons in dict_product(constraints):
    args['tau']=cons['tau']
    args['task']=cons['task']
    runs = {}
    for rx in [1,100]:
        for irb, rb in enumerate([0.01,100]):
            if rx==1 and irb>0: continue
            args['rx'], args['rb'] = rx, rb
            if model=="nn":
                run = run_nn(args)
            elif model=="rf":
                run = run_rf(args)
            runs[(rx,rb)]=run
    if len(args['Psi1_list'])>1:
        title = datadir+'parameter_wise_{}_{}.pyT'.format(args['task'], args['tau'])
    else:
        title = datadir+   'sample_wise_{}_{}.pyT'.format(args['task'], args['tau'])
    torch.save(runs, title)

# Real data

In [ ]:
fig, axarr = plt.subplots(1,2,figsize=(9,4))
for i, dataset in enumerate(['CIFAR10','MNIST']):
    tr_data, te_data = get_dataset(dataset, D=64, N_train=10000, N_test=10000, resize_method='resize')
    data = tr_data
    cov = (data-data.mean(dim=0)).t() @ (data-data.mean(dim=0)) / len(data)
    spectrum = torch.symeig(cov)[0]
    sigma = spectrum/spectrum.sum()
    entropy = -torch.sum(sigma * torch.log10(sigma))
    erank = 10**(entropy)
    print(erank)
    hist = axarr[i].hist(spectrum, alpha=.5, bins=20)
    axarr[i].set_title('{0}, Effective rank = {1:.1f}'.format(dataset, erank.item()))
plt.tight_layout()
plt.show()

ds, cs = hist[0], hist[1]
print(ds,cs)
ds /= ds.sum()
cs = (cs[1:]+cs[:-1])/2
torch.save((ds, cs), datadir+'histogram.pyT')

In [ ]:
args = {'D' :100,
    'Psi1_list'  : [2],
    'Psi2_list' : np.logspace(-1,1,19),
    'N_test' : 1000,
    'K' : 5,
    'lam' : 1e-3,
    'actname': 'tanh',
    'dataset' : 'CIFAR10',
    'd': 1,
    'c': [1,1],
    'b': [1,1]}

model = "rf"

for tau in [0,.1]:
    runs = []
    args['tau']=tau
        
    if model=="nn":
        run = run_nn(args)
    elif model=="rf":
        run = run_rf(args)
    runs.append(run)
        
    if len(args['Psi1_list'])>1:
        title = datadir+'parameter_wise_{}_{}_{}.pyT'.format(model, tau, args['dataset'])
    else: 
        title = datadir+   'sample_wise_{}_{}_{}.pyT'.format(model, tau, args['dataset'])
    torch.save(runs, title)

In [ ]:
model = 'rf'
plot_type = 'sample_wise'

if plot_type not in ['parameter_wise', 'sample_wise']:
    raise 
    
plt.figure(figsize=(8,5))

for tau in [0, .1]:
    runs = torch.load(datadir+plot_type+'_{}_{}_CIFAR10.pyT'.format(model, tau))
    for i,run in enumerate(runs):
        args=run['args']
        x = args['Psi1_list'] if plot_type=='parameter_wise' else args['Psi2_list']
        plt.plot(x, run['te_errors'], label=r'$\Delta={}$'.format(args['tau']))
plt.xscale('log')
plt.yscale('log')
#plt.axvline(x=args['Psi1_list'£], color='grey', label='N=P', ls='--')
if plot_type == 'parameter_wise':
    plt.xlabel('$p/d$')
else: 
    plt.xlabel('$m/d$')

plt.ylabel('Test loss')
plt.legend()

plt.tight_layout()
plt.savefig(savedir+plot_type+'_{}_{}.pdf'.format(model,tau), bbox_inches='tight')
plt.show()
